<a href="https://colab.research.google.com/github/aswinaus/LLM_Inference/blob/main/mistral_quant_awq_load_local.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/huggingface/transformers torch accelerate langchain langchain_huggingface datasets

Code is essentially forcing Python to always use "UTF-8" as the preferred encoding, regardless of the user's actual system settings. UTF-8 is a widely used encoding that can represent a vast range of characters from different languages. By enforcing UTF-8, you can help ensure that your code works consistently across different platforms and avoids encoding-related errors. It's a common practice for improving compatibility and preventing issues with text handling in Python programs.

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
from google.colab import userdata
HUGGING_FACE_TOKEN = userdata.get('HUGGING_FACE_TOKEN')

In [ ]:
!huggingface-cli login --token $HUGGING_FACE_TOKEN

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `Agents` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `Agents`


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# Download Data
data_dir = '/content/drive/MyDrive'

Mounted at /content/drive


In [ ]:
# Import libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
import transformers
import torch
from langchain_huggingface import HuggingFacePipeline
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
from threading import Thread

The nvidia-smi command is a utility provided by NVIDIA to query and display information about your NVIDIA GPU(s) (Graphics Processing Unit). This includes things like:

GPU model and name
Driver version
GPU utilization
Memory usage
Temperature
Power consumption
Processes running on the GPU

In [ ]:
!nvidia-smi

Sat Feb  8 20:27:34 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P8             11W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
import textwrap

def wrap_text(text, width=90): #preserve_newlines
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

In [1]:
!pip install autoawq
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 91.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 70.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.7 MB/s eta 0:00:00

In [ ]:
from typing import Tuple, Optional, Union, Dict, Any
from transformers import PreTrainedModel, AutoModel, AutoTokenizer, AutoConfig
from transformers.tokenization_utils_base import PreTrainedTokenizerBase

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
data_dir = '/content/drive/MyDrive' # Input a data dir path from your mounted Google Drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
quant_path = f"/{data_dir}/LLMs/Mistral/Mistral-Small-24B-Instruct-2501"

In [ ]:
local_model_path = quant_path
local_tokenizer = AutoTokenizer.from_pretrained(quant_path)
local_model = AutoModel.from_pretrained(quant_path,low_cpu_mem_usage=True)

You have loaded an AWQ model on CPU and have a CUDA device available, make sure to set your model on a GPU device in order to run your model.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

local_model.to(device) moves all the model's parameters and buffers to the specified device (in this case, device, which is set to 'cuda' if a GPU is available). Deep learning models often have a large number of parameters and require significant computational power. GPUs are designed for parallel processing and can significantly speed up the training and inference of deep learning models. By moving the model to the GPU, you leverage its computational capabilities for faster execution.

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
local_model.to(device)

MistralModel(
  (embed_tokens): Embedding(131072, 5120)
  (layers): ModuleList(
    (0-39): 40 x MistralDecoderLayer(
      (self_attn): MistralSdpaAttention(
        (q_proj): WQLinear_GEMM(in_features=5120, out_features=4096, bias=False, w_bit=4, group_size=128)
        (k_proj): WQLinear_GEMM(in_features=5120, out_features=1024, bias=False, w_bit=4, group_size=128)
        (v_proj): WQLinear_GEMM(in_features=5120, out_features=1024, bias=False, w_bit=4, group_size=128)
        (o_proj): WQLinear_GEMM(in_features=4096, out_features=5120, bias=False, w_bit=4, group_size=128)
        (rotary_emb): MistralRotaryEmbedding()
      )
      (mlp): MistralMLP(
        (gate_proj): WQLinear_GEMM(in_features=5120, out_features=32768, bias=False, w_bit=4, group_size=128)
        (up_proj): WQLinear_GEMM(in_features=5120, out_features=32768, bias=False, w_bit=4, group_size=128)
        (down_proj): WQLinear_GEMM(in_features=32768, out_features=5120, bias=False, w_bit=4, group_size=128)
        (

In [ ]:
from datasets import load_dataset
import pandas as pd
dataset = load_dataset("aswinaus/tax_statistics_dataset_by_income_range", download_mode="force_redownload")
df=pd.DataFrame(dataset['train'])
df.head(10)

README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

(…)x_statistics_dataset_by_income_range.csv:   0%|          | 0.00/272k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/303 [00:00<?, ? examples/s]

,STATEFIPS,STATE,zipcode,Size of adjusted gross income,No of returns,No of single returns,No of joint returns,No of head of household returns,Number of electronically filed returns,Number of computer prepared paper returns,...,Number of returns with net investment income tax,Net investment income tax amount,Number of returns with tax due at time of filing,Tax due at time of filing amount,Number of returns with total overpayments,Total overpayments amount,Number of returns with overpayments refunded,Overpayments refunded amount,Number of returns with credit to next years estimated tax,Credited to next years estimated tax amount
0,1,AL,0,"$1 under $25,000",778210,491030,84770,189600,712890,30670,...,0,0,62720,51936,671860,1700965,669570,1694792,1980,3512
1,1,AL,0,"$25,000 under $50,000",525940,247140,123910,139860,481760,18960,...,0,0,85860,122569,438020,1274802,435210,1266557,3670,7410
2,1,AL,0,"$50,000 under $75,000",285700,105140,128140,44560,260570,10670,...,0,0,73980,154932,212040,575315,208470,564202,5020,13653
3,1,AL,0,"$75,000 under $100,000",179070,38820,123110,13740,164300,5020,...,0,0,51330,139065,126850,401581,123310,388749,3040,10377
4,1,AL,0,"$100,000 under $200,000",257010,28180,216740,7150,236850,8400,...,90,141,104290,460071,152790,598248,144640,539385,9180,56257
5,1,AL,0,"$200,000 or more",74810,4540,66580,530,69330,1760,...,39430,141243,40030,946610,32060,681565,22420,257633,9450,372747
6,2,AK,0,"$1 under $25,000",102820,81530,7850,10850,89210,6360,...,0,0,17720,8643,75910,133915,75430,132166,370,670
7,2,AK,0,"$25,000 under $50,000",79910,48680,14900,13510,72490,3260,...,0,0,11540,19564,68020,185024,67520,182576,740,1890
8,2,AK,0,"$50,000 under $75,000",51890,25460,17880,6840,46570,2160,...,0,0,11330,26920,40310,118691,39720,115674,830,2922
9,2,AK,0,"$75,000 under $100,000",36350,12880,19390,3190,32620,1630,...,0,0,9690,29647,26340,90982,25670,87439,680,2366


In [ ]:
from transformers import TrainingArguments, Trainer

# 1. Data Preparation (Example - adjust to your specific task)
def preprocess_function(examples):
    # Tokenize income ranges and tax statistics
    inputs = local_tokenizer(examples["income_range"], max_length=128, truncation=True,
                             padding="max_length", return_tensors="pt")
    # ... further processing (e.g., converting tax statistics to numerical labels)
    return inputs

# Apply preprocessing to the DataFrame
df = df.map(preprocess_function)

# 2. Model Adaptation (Fine-tuning example)
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,  # Adjust as needed
    # ... other training arguments
)

trainer = Trainer(
    model=local_model,
    args=training_args,
    train_dataset=df["train"],  # Assuming you have a train split
    eval_dataset=df["validation"], # Assuming you have a validation split
    # ... data collator, compute_metrics if needed
)

# 3. Training
trainer.train()

# 4. Evaluation (example)
eval_results = trainer.evaluate()
print(eval_results)

TypeError: 'int' object is not subscriptable